In [2]:
from setup import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import RandomForestClassifier
import glob

df = pd.read_csv('../_LeagueofLegends.csv')
df_temp = df[['blueTopChamp','blueJungleChamp','blueMiddleChamp','blueADCChamp',
           'blueSupportChamp','redTopChamp','redJungleChamp','redMiddleChamp','redADCChamp',
           'redSupportChamp']]
list_champions = df_temp.stack().unique()
len(list_champions)

df['blueBans'].replace('[^A-Za-z\s]+','',regex=True,inplace=True)
df['redBans'].replace('[^A-Za-z\s]+','',regex=True,inplace=True)
li_temp_red = [x for x in df['redBans'].str.split()]
li_temp_blue = [x for x in df['blueBans'].str.split()]
li_total_bans = []
for x in range(len(li_temp_red)):
   li_total_bans.append(list(set().union(li_temp_red[x],li_temp_blue[x])))
df['totalBans'] = li_total_bans

df_2015 = df[df['Year']==2015]
df_2016 = df[df['Year']==2016]
df_2017 = df[df['Year']==2017]
df_2015['isTrain'] = np.random.uniform(0, 1, len(df_2015)) <= .75
df_2016['isTrain'] = np.random.uniform(0, 1, len(df_2016)) <= .75
df_2017['isTrain'] = np.random.uniform(0, 1, len(df_2017)) <= .75

blueTeam_2015 = df_2015[['blueTeamTag','blueTop','blueTopChamp','blueJungle','blueJungleChamp',
                        'blueMiddle','blueMiddleChamp','blueADC','blueADCChamp','blueSupport',
                       'blueSupportChamp','totalBans','isTrain']]
blueTeam_2015.reset_index(drop=True)
redTeam_2015 = df_2015[['redTeamTag','redTop','redTopChamp','redJungle','redJungleChamp',
                        'redMiddle','redMiddleChamp','redADC','redADCChamp','redSupport',
                       'redSupportChamp','totalBans','isTrain']]
redTeam_2015.reset_index(drop=True)
blueTeam_2016 = df_2016[['blueTeamTag','blueTop','blueTopChamp','blueJungle','blueJungleChamp',
                        'blueMiddle','blueMiddleChamp','blueADC','blueADCChamp','blueSupport',
                       'blueSupportChamp','totalBans','isTrain']]
blueTeam_2016.reset_index(drop=True)

C:\Users\sw_hutyra\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\sw_hutyra\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\sw_hutyra\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

,blueTeamTag,blueTop,blueTopChamp,blueJungle,blueJungleChamp,blueMiddle,blueMiddleChamp,blueADC,blueADCChamp,blueSupport,blueSupportChamp,totalBans,isTrain
0,TSM,Hauntzer,DrMundo,Svenskeren,Elise,Bjergsen,TwistedFate,Doublelift,Tristana,Yellowstar,Alistar,"[Gangplank, Olaf, Fiora, TahmKench, Ryze, Corki]",True
1,C9,Balls,Ryze,Rush,LeeSin,Jensen,Ekko,Sneaky,MissFortune,Bunny FuFuu,Braum,"[Gangplank, Alistar, Lissandra, Lulu, Fiora, T...",True
2,DIG,Smittyj,Malphite,Kirei,Kindred,Shiphtur,Anivia,Apollo,Ezreal,KiWiKiD,Alistar,"[Gangplank, Elise, Lulu, RekSai, TahmKench, Ryze]",True
3,REN,RF Legendary,DrMundo,Crumbz,RekSai,AlexIch,Orianna,Freeze,Kalista,Remi,Alistar,"[Gangplank, Thresh, Lulu, TahmKench, Ryze, Kin...",True
4,FOX,kfo,Lulu,Hard,Elise,Froggen,Lissandra,Keith,Lucian,Big,Thresh,"[Gangplank, Graves, Nidalee, Anivia, Fiora, Ryze]",False
5,REN,RF Legendary,DrMundo,Crumbz,RekSai,AlexIch,Ryze,Freeze,Lucian,Remi,Morgana,"[Gangplank, Alistar, Thresh, Lulu, TwistedFate...",True
6,DIG,Smittyj,Fiora,Kirei,Elise,Shiphtur,Ahri,Apollo,Ezreal,KiWiKiD,Trundle,"[Gangplank, Lulu, RekSai, TwistedFate, TahmKen...",True
7,TSM,Hauntzer,TahmKench,Svenskeren,RekSai,Bjergsen,Syndra,Doublelift,Ezreal,Yellowstar,Bard,"[Gangplank, Graves, Lissandra, Lulu, Ryze, Kal...",True
8,FOX,kfo,Lissandra,Hard,RekSai,Froggen,Anivia,Keith,MissFortune,Big,Trundle,"[Gangplank, Nidalee, Lulu, TahmKench, Ryze, Lu...",False
9,IMT,Huni,Fiora,Reignover,Elise,Pobelter,Lulu,WildTurtle,Corki,Adrian,Janna,"[Gangplank, Graves, RekSai, TahmKench, Poppy, ...",True


In [153]:
from setup import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import RandomForestClassifier
import glob

pd.set_option('display.max_rows', 30)
df = pd.read_csv('../_LeagueofLegends.csv')
display(df.columns)
blue_df = df[['blueTopChamp','blueJungleChamp','blueMiddleChamp','blueADCChamp','blueSupportChamp']]
blue_df.columns = ['Top','Jungle','Mid','ADC','Support']
red_df = df[['redTopChamp','redJungleChamp','redMiddleChamp','redADCChamp','redSupportChamp']]
red_df.columns = ['Top','Jungle','Mid','ADC','Support']
red_blue_df = pd.concat([red_df,blue_df]).reset_index(drop=True)
red_blue_df = red_blue_df.apply(lambda g: g.str.lower())

top_count = red_blue_df.groupby('Top').apply(lambda g: g.count())['Top'].to_frame()
jun_count = red_blue_df.groupby('Jungle').apply(lambda g: g.count())['Jungle'].to_frame()
mid_count = red_blue_df.groupby('Mid').apply(lambda g: g.count())['Mid'].to_frame()
adc_count = red_blue_df.groupby('ADC').apply(lambda g: g.count())['ADC'].to_frame()
sup_count = red_blue_df.groupby('Support').apply(lambda g: g.count())['Support'].to_frame()
top_count.index.names=['Champion']
jun_count.index.names=['Champion']
mid_count.index.names=['Champion']
adc_count.index.names=['Champion']
sup_count.index.names=['Champion']
# display(mid_count)
counts = pd.merge(top_count,jun_count,left_index=True,right_index=True,how='outer')
counts = pd.merge(counts,mid_count,left_index=True,right_index=True,how='outer')
counts = pd.merge(counts,adc_count,left_index=True,right_index=True,how='outer')
counts = pd.merge(counts,sup_count,left_index=True,right_index=True,how='outer')
pd.set_option('display.max_rows', -1)
counts = counts.fillna(0).astype('int')
display(counts)
counts.to_html('champ_counts.html')

Index(['MatchHistory', 'League', 'Season', 'Year', 'blueTeamTag', 'bResult',
       'rResult', 'redTeamTag', 'gamelength', 'golddiff', 'goldblue', 'bKills',
       'bTowers', 'bInhibs', 'bDragons', 'bBarons', 'bHeralds', 'goldred',
       'rKills', 'rTowers', 'rInhibs', 'rDragons', 'rBarons', 'rHeralds',
       'blueTop', 'blueTopChamp', 'goldblueTop', 'blueJungle',
       'blueJungleChamp', 'goldblueJungle', 'blueMiddle', 'blueMiddleChamp',
       'goldblueMiddle', 'blueADC', 'blueADCChamp', 'goldblueADC',
       'blueSupport', 'blueSupportChamp', 'goldblueSupport', 'blueBans',
       'redTop', 'redTopChamp', 'goldredTop', 'redJungle', 'redJungleChamp',
       'goldredJungle', 'redMiddle', 'redMiddleChamp', 'goldredMiddle',
       'redADC', 'redADCChamp', 'goldredADC', 'redSupportChamp', 'redSupport',
       'goldredSupport', 'redBans'],
      dtype='object')

,Top,Jungle,Mid,ADC,Support
Champion,,,,,
aatrox,0,1,0,0,0
ahri,0,0,343,0,0
akali,3,0,0,0,0
alistar,4,1,0,0,974
amumu,0,4,0,0,0
anivia,0,0,82,0,0
annie,0,0,0,0,238
ashe,0,0,0,854,0
aurelionsol,0,0,51,0,0


In [45]:
from setup import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import RandomForestClassifier
import re
import glob


In [39]:
df = pd.read_csv('../_LeagueofLegends.csv')

In [18]:
display(df)

,MatchHistory,League,Season,Year,blueTeamTag,bResult,rResult,redTeamTag,gamelength,golddiff,...,redMiddle,redMiddleChamp,goldredMiddle,redADC,redADCChamp,goldredADC,redSupportChamp,redSupport,goldredSupport,redBans
0,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,TSM,1,0,C9,40,"[0, 0, -14, -65, -268, -431, -488, -789, -494,...",...,Hai,Fizz,"[475, 475, 552, 842, 1178, 1378, 1635, 1949, 2...",Sneaky,Sivir,"[475, 475, 532, 762, 1097, 1469, 1726, 2112, 2...",Thresh,LemonNation,"[515, 515, 577, 722, 911, 1042, 1194, 1370, 14...","['Tristana', 'Leblanc', 'Nidalee']"
1,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,CST,0,1,DIG,38,"[0, 0, -26, -18, 147, 237, -152, 18, 88, -242,...",...,Shiphtur,Azir,"[475, 475, 552, 786, 1097, 1389, 1660, 1955, 2...",CoreJJ,Corki,"[475, 475, 532, 868, 1220, 1445, 1732, 1979, 2...",Annie,KiWiKiD,"[515, 515, 583, 752, 900, 1066, 1236, 1417, 15...","['RekSai', 'Janna', 'Leblanc']"
2,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,WFX,1,0,GV,40,"[0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ...",...,Keane,Azir,"[475, 475, 533, 801, 1006, 1233, 1385, 1720, 1...",Cop,Corki,"[475, 475, 533, 781, 1085, 1398, 1782, 1957, 2...",Janna,BunnyFuFuu,"[515, 515, 584, 721, 858, 1002, 1168, 1303, 14...","['Leblanc', 'Zed', 'RekSai']"
3,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,TIP,0,1,TL,41,"[0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ...",...,Fenix,Lulu,"[475, 475, 532, 771, 1046, 1288, 1534, 1776, 2...",KEITH,KogMaw,"[475, 475, 532, 766, 1161, 1438, 1776, 1936, 2...",Janna,Xpecial,"[515, 515, 583, 721, 870, 1059, 1205, 1342, 15...","['RekSai', 'Rumble', 'LeeSin']"
4,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,CLG,1,0,T8,35,"[40, 40, 44, -36, 113, 158, -121, -191, 23, 20...",...,Slooshi8,Lulu,"[475, 475, 532, 807, 1042, 1338, 1646, 1951, 2...",Maplestreet8,Corki,"[475, 475, 532, 792, 1187, 1488, 1832, 2136, 2...",Annie,Dodo8,"[475, 475, 538, 671, 817, 948, 1104, 1240, 136...","['Rumble', 'Sivir', 'Rengar']"
5,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,DIG,0,1,TIP,24,"[0, 0, 20, -42, -48, -24, -219, -272, -319, -5...",...,XiaoWeiXiao,Lulu,"[475, 475, 532, 821, 1152, 1493, 1719, 2123, 2...",Apollo,Corki,"[475, 475, 532, 826, 1107, 1422, 1760, 2081, 2...",Thresh,Adrian,"[515, 515, 577, 749, 926, 1081, 1235, 1360, 15...","['Azir', 'RekSai', 'Lissandra']"
6,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,CST,1,0,WFX,39,"[0, 13, -7, 6, -353, -215, -795, -764, 369, 35...",...,Pobelter,Leblanc,"[475, 475, 553, 856, 1157, 1433, 1670, 1990, 2...",Altec,Tristana,"[475, 475, 533, 771, 1102, 1444, 2126, 2450, 2...",Nami,Gleeb,"[515, 520, 589, 764, 937, 1089, 1367, 1509, 16...","['RekSai', 'Gnar', 'Kassadin']"
7,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,TL,1,0,CLG,43,"[0, 0, 26, 91, -109, 264, 178, 66, -65, 206, 5...",...,Link,Leblanc,"[475, 475, 532, 821, 1168, 1393, 1594, 1871, 2...",Doublelift,Sivir,"[475, 475, 532, 761, 1107, 1478, 1746, 2126, 2...",Blitzcrank,Aphromoo,"[515, 515, 577, 718, 871, 995, 1149, 1320, 149...","['Nunu', 'Gnar', 'RekSai']"
8,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,C9,0,1,GV,41,"[0, -10, 0, 162, -269, -63, 323, 225, 274, 624...",...,Keane,Zed,"[475, 475, 532, 786, 1052, 1297, 1539, 1898, 2...",Cop,Corki,"[475, 475, 532, 746, 1243, 1610, 1867, 2175, 2...",Morgana,Bunny FuFuu,"[515, 525, 593, 749, 1349, 1504, 1664, 1886, 2...","['Rumble', 'Lissandra', 'Fizz']"
9,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,T8,1,0,TSM,32,"[0, 0, 0, 68, 498, 210, 283, 311, 133, 291, 32...",...,Bjergsen,Leblanc,"[475, 475, 532, 856, 1157, 1432, 1700, 2086, 2...",WildTurtle,Graves,"[475, 475, 532, 796, 1147, 1432, 1684, 1972, 2...",Morgana,Lustbo

In [42]:
df['Game'] = df['MatchHistory'].str.replace(r'http://matchhistory.na.leagueoflegends.com/en/*/#match-details/.*/','')
df['Game'] = df['Game'].str.replace(r'\?gameHash=.*','')
display(df)

,MatchHistory,League,Season,Year,blueTeamTag,bResult,rResult,redTeamTag,gamelength,golddiff,...,redMiddleChamp,goldredMiddle,redADC,redADCChamp,goldredADC,redSupportChamp,redSupport,goldredSupport,redBans,Game
0,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,TSM,1,0,C9,40,"[0, 0, -14, -65, -268, -431, -488, -789, -494,...",...,Fizz,"[475, 475, 552, 842, 1178, 1378, 1635, 1949, 2...",Sneaky,Sivir,"[475, 475, 532, 762, 1097, 1469, 1726, 2112, 2...",Thresh,LemonNation,"[515, 515, 577, 722, 911, 1042, 1194, 1370, 14...","['Tristana', 'Leblanc', 'Nidalee']",30030
1,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,CST,0,1,DIG,38,"[0, 0, -26, -18, 147, 237, -152, 18, 88, -242,...",...,Azir,"[475, 475, 552, 786, 1097, 1389, 1660, 1955, 2...",CoreJJ,Corki,"[475, 475, 532, 868, 1220, 1445, 1732, 1979, 2...",Annie,KiWiKiD,"[515, 515, 583, 752, 900, 1066, 1236, 1417, 15...","['RekSai', 'Janna', 'Leblanc']",30054
2,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,WFX,1,0,GV,40,"[0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ...",...,Azir,"[475, 475, 533, 801, 1006, 1233, 1385, 1720, 1...",Cop,Corki,"[475, 475, 533, 781, 1085, 1398, 1782, 1957, 2...",Janna,BunnyFuFuu,"[515, 515, 584, 721, 858, 1002, 1168, 1303, 14...","['Leblanc', 'Zed', 'RekSai']",30067
3,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,TIP,0,1,TL,41,"[0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ...",...,Lulu,"[475, 475, 532, 771, 1046, 1288, 1534, 1776, 2...",KEITH,KogMaw,"[475, 475, 532, 766, 1161, 1438, 1776, 1936, 2...",Janna,Xpecial,"[515, 515, 583, 721, 870, 1059, 1205, 1342, 15...","['RekSai', 'Rumble', 'LeeSin']",30091
4,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,CLG,1,0,T8,35,"[40, 40, 44, -36, 113, 158, -121, -191, 23, 20...",...,Lulu,"[475, 475, 532, 807, 1042, 1338, 1646, 1951, 2...",Maplestreet8,Corki,"[475, 475, 532, 792, 1187, 1488, 1832, 2136, 2...",Annie,Dodo8,"[475, 475, 538, 671, 817, 948, 1104, 1240, 136...","['Rumble', 'Sivir', 'Rengar']",30105
5,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,DIG,0,1,TIP,24,"[0, 0, 20, -42, -48, -24, -219, -272, -319, -5...",...,Lulu,"[475, 475, 532, 821, 1152, 1493, 1719, 2123, 2...",Apollo,Corki,"[475, 475, 532, 826, 1107, 1422, 1760, 2081, 2...",Thresh,Adrian,"[515, 515, 577, 749, 926, 1081, 1235, 1360, 15...","['Azir', 'RekSai', 'Lissandra']",30131
6,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,CST,1,0,WFX,39,"[0, 13, -7, 6, -353, -215, -795, -764, 369, 35...",...,Leblanc,"[475, 475, 553, 856, 1157, 1433, 1670, 1990, 2...",Altec,Tristana,"[475, 475, 533, 771, 1102, 1444, 2126, 2450, 2...",Nami,Gleeb,"[515, 520, 589, 764, 937, 1089, 1367, 1509, 16...","['RekSai', 'Gnar', 'Kassadin']",30133
7,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,TL,1,0,CLG,43,"[0, 0, 26, 91, -109, 264, 178, 66, -65, 206, 5...",...,Leblanc,"[475, 475, 532, 821, 1168, 1393, 1594, 1871, 2...",Doublelift,Sivir,"[475, 475, 532, 761, 1107, 1478, 1746, 2126, 2...",Blitzcrank,Aphromoo,"[515, 515, 577, 718, 871, 995, 1149, 1320, 149...","['Nunu', 'Gnar', 'RekSai']",30149
8,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,C9,0,1,GV,41,"[0, -10, 0, 162, -269, -63, 323, 225, 274, 624...",...,Zed,"[475, 475, 532, 786, 1052, 1297, 1539, 1898, 2...",Cop,Corki,"[475, 475, 532, 746, 1243, 1610, 1867, 2175, 2...",Morgana,Bunny FuFuu,"[515, 525, 593, 749, 1349, 1504, 1664, 1886, 2...","['Rumble', 'Lissandra', 'Fizz']",30161
9,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,T8,1,0,TSM,32,"[0, 0, 0, 68, 498, 210, 283, 311, 133, 291, 32...",...,Leblanc,"[475, 475, 532, 856, 1157, 1432, 1700, 2086, 2...",WildTurtle,Graves,"[475, 475, 532, 796, 1147, 1432, 1684, 1972, 2...",Morgana,Lustboy,"[515, 515, 583, 722, 86

In [52]:
stat_list = glob.glob('Compiled_Stats/*.csv')
new_stat_list = []
# display(stat_list)
for x in stat_list:
    x = x.replace('Compiled_Stats\\','')
    x = x.replace('.csv','')
    new_stat_list.append(x)
display(new_stat_list)

['Assists',
 'Control_Wards_Purchased',
 'Damage_Healed',
 'Damage_Taken',
 'Deaths',
 'First_Blood',
 'Gold_Earned',
 'Gold_Spent',
 'Kills',
 'Largest_Critical_Strike',
 'Largest_Killing_Spree',
 'Largest_Multi_Kill',
 'Magic_Damage_Dealt',
 'Magic_Damage_Taken',
 'Magic_Damage_to_Champions',
 'Minions_Killed',
 'Neutral_Minions_Killed',
 'Neutral_Minions_Killed_in_Enemy_Jungle',
 "Neutral_Minions_Killed_in_Team's_Jungle",
 'Physical_Damage_Dealt',
 'Physical_Damage_Taken',
 'Physical_Damage_to_Champions',
 'Stealth_Wards_Purchased',
 'Total_Damage_Dealt',
 'Total_Damage_to_Champions',
 'Total_Damage_to_Objectives',
 'Total_Damage_to_Turrets',
 'True_Damage_Dealt',
 'True_Damage_Taken',
 'True_Damage_to_Champions',
 'Wards_Destroyed',
 'Wards_Placed']

In [79]:
bans = df[['Game','Year','blueBans','redBans']]
def sub_na(ls):
    if type(ls) is type(str):
        ls = [ls,'NaN']
    else:
        while len(ls) < 3:
            ls.append('Null')        
    return ls
bans['newBlueBans']=bans['blueBans'].map(lambda g : sub_na(g))
display(bans)

AttributeError: 'str' object has no attribute 'append'